<a href="https://colab.research.google.com/github/adrian-str/Hands-on_ML/blob/main/Chapter7_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load MNIST dataset 
### From Kaggle

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [7]:
y_train=train["label"].to_numpy()
X_train=train.drop("label",axis=1).to_numpy()
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
# del train 
# del test

In [8]:
X_train,y_train,X_test,y_test,X_val,y_val=X_train_scaled[:30000],y_train[:30000],X_train_scaled[30000:36000],y_train[30000:36000],X_train_scaled[36000:],y_train[36000:]

## Random Forest & Extra Trees

In [10]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
rf_clf=RandomForestClassifier(random_state=42,n_jobs=-1)
et_clf=ExtraTreesClassifier(random_state=42,n_jobs=-1)
svm_clf=SVC(random_state=42,kernel="rbf",probability=True)

In [11]:
rf_clf.fit(X_train,y_train)
et_clf.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)

## SVM

In [12]:
svm_clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [14]:
# from sklearn.metrics import accuracy_score
# y_pred_s=svm_clf.predict(X_val)
print("SVM: ",accuracy_score(y_true=y_val,y_pred=y_pred_s))

SVM:  0.9571666666666667


## Voting

In [15]:
from sklearn.ensemble import VotingClassifier
voting_clf_soft=VotingClassifier( estimators=[('svc',svm_clf),('rf',rf_clf),('et',et_clf)],voting='soft',n_jobs=-1)
voting_clf_hard=VotingClassifier( estimators=[('svc',svm_clf),('rf',rf_clf),('et',et_clf)],voting='hard',n_jobs=-1)

In [17]:
# voting_clf_soft.fit(X_train,y_train)
# y_pred_v=voting_clf_soft.predict(X_val)
print("Voting classifier (soft): ",accuracy_score(y_true=y_val,y_pred=y_pred_v))

Voting classifier (soft):  0.971


## Stacking

In [18]:
y_pred_r=rf_clf.predict(X_val)
y_pred_e=et_clf.predict(X_val)
# y_pred_s=svm_clf.predict(X_val)
Xi=np.c_[y_pred_s,y_pred_r,y_pred_e,y_pred_v]

In [20]:
y_pred_r=rf_clf.predict(X_test)
y_pred_e=et_clf.predict(X_test)
y_pred_s=svm_clf.predict(X_test)
y_pred_v=voting_clf_soft.predict(X_test)
Xt=np.c_[y_pred_s,y_pred_r,y_pred_e,y_pred_v]

In [19]:
from sklearn.tree import DecisionTreeClassifier
dt_clf=DecisionTreeClassifier()
dt_clf.fit(Xi,y_val)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
y_pred=dt_clf.predict(Xt)
accuracy_score(y_true=y_test,y_pred=y_pred)

0.9728333333333333